In [1]:
import pandas as pd
import re

In [2]:
bank_statement_df = pd.read_excel(r"D:\NPS_phase1\nps-reconciliation\output\BANK STMT PHASE 4\Bank_Shangrila.xlsx",skiprows=1)

In [3]:
bank_statement_df.head()

,S.N,Ac.Number,Amount,Txn Type,Desc1,Desc2,Desc3,Date,Number
0,1,9105801120601,4426426.28,C,2301230829IGW00B,eod8-28,NPS CENTRAL SETTLEME,2023-08-29T12:53:00,NaN
1,2,9105801120601,10000.00,C,9841034054,"MOB,100003989053",1648192,2023-08-29T12:29:00,NaN
2,3,9105801120601,20000.00,C,9816642631,"MOB,100003989010",1648092,2023-08-29T12:20:00,NaN
3,4,9105801120601,15000.00,C,9851086526,"MOB,100003988851",1647903,2023-08-29T12:01:00,NaN
4,5,9105801120601,200.00,C,9748361397,"MOB,100003988734",1647700,2023-08-29T11:41:00,NaN


In [4]:
for index, row in bank_statement_df.iterrows():
    # print(bank_statement_df.iloc[index]['Transaction ID'])
    for roww in (row['Desc2'] , row['Desc3']):
        id = ''
        if "NPS-IF-" in roww:
            id = re.findall(r'[0-9]{7}',roww)
            bank_statement_df.loc[index, 'Transaction Id'] = id[0]
            break
        
        
        elif "10000" in roww:
            if re.findall(r'#10*\d{7}',roww):
                continue
            id = re.findall(r'10*[0-9]{7}',roww)
            bank_statement_df.loc[index, 'Transaction Id'] = id[0].replace("10000","")
            break
        
        elif "FTMS-" in roww:
            id = re.findall(r'[0-9]{6}',roww)
            bank_statement_df.loc[index, 'Transaction Id'] = id[0]

In [6]:
bank_statement_df.head(15)

,S.N,Ac.Number,Amount,Txn Type,Desc1,Desc2,Desc3,Date,Number,Transaction Id
0,1,9105801120601,4426426.28,C,2301230829IGW00B,eod8-28,NPS CENTRAL SETTLEME,2023-08-29T12:53:00,NaN,NaN
1,2,9105801120601,10000.00,C,9841034054,"MOB,100003989053",1648192,2023-08-29T12:29:00,NaN,3989053
2,3,9105801120601,20000.00,C,9816642631,"MOB,100003989010",1648092,2023-08-29T12:20:00,NaN,3989010
3,4,9105801120601,15000.00,C,9851086526,"MOB,100003988851",1647903,2023-08-29T12:01:00,NaN,3988851
4,5,9105801120601,200.00,C,9748361397,"MOB,100003988734",1647700,2023-08-29T11:41:00,NaN,3988734
5,6,9105801120601,30940.00,D,LAXMIBANK#cash,020447,NPS-IF-3142883Off,2023-08-29T23:40:00,NaN,3142883
6,7,9105801120601,1000.00,C,9762525844,"MOB,100003988698",1647664,2023-08-29T11:38:00,NaN,3988698
7,8,9105801120601,200166.95,D,PRABHUMONEY#PMTFT23,020423,NPS-IF-3136924,2023-08-29T10:56:00,NaN,3136924
8,9,9105801120601,10087.00,D,PRABHUMONEY#PMT1138,020422,NPS-IF-3136891,2023-08-29T10:51:00,NaN,3136891
9,10,9105801120601,50000.00,C,9842100514,"MOB,100003988389",1647160,2023-08-29T10:48:00,NaN,3988389


In [9]:
bank_statement_df['Transaction Id'].count()

54

In [10]:
def standard_format_phase_4(row):
        """
        Determine transaction mode and amount from bank statement data.

        Parameters:
            row (Series): A row of bank statement data.

        Returns:
            Series: Mode (CR/DR) and amount.
        """
        if row['Txn Type'] == 'D' :
            mode = 'DR'
            amount = abs(row['Amount'])
        elif row['Txn Type'] == 'C':
            mode = 'CR'
            amount = abs(row['Amount'])
        else:
            mode = None
            amount = None
        return pd.Series([mode, amount], index=['Mode', 'Amount'])

In [11]:
bank_statement_df[['Mode', 'Amount']] = bank_statement_df.apply(standard_format_phase_4, axis=1)


In [12]:
bank_statement_df.head()

,S.N,Ac.Number,Amount,Txn Type,Desc1,Desc2,Desc3,Date,Number,Transaction Id,Mode
0,1,9105801120601,4426426.28,C,2301230829IGW00B,eod8-28,NPS CENTRAL SETTLEME,2023-08-29T12:53:00,NaN,NaN,CR
1,2,9105801120601,10000.00,C,9841034054,"MOB,100003989053",1648192,2023-08-29T12:29:00,NaN,3989053,CR
2,3,9105801120601,20000.00,C,9816642631,"MOB,100003989010",1648092,2023-08-29T12:20:00,NaN,3989010,CR
3,4,9105801120601,15000.00,C,9851086526,"MOB,100003988851",1647903,2023-08-29T12:01:00,NaN,3988851,CR
4,5,9105801120601,200.00,C,9748361397,"MOB,100003988734",1647700,2023-08-29T11:41:00,NaN,3988734,CR


In [13]:
bank_statement_df['Date'] = pd.to_datetime(bank_statement_df['Date'], format="%Y-%m-%dT%H:%M:%S").dt.date

In [14]:
bank_statement_df

,S.N,Ac.Number,Amount,Txn Type,Desc1,Desc2,Desc3,Date,Number,Transaction Id,Mode
0,1,9105801120601,4426426.28,C,2301230829IGW00B,eod8-28,NPS CENTRAL SETTLEME,2023-08-29,NaN,NaN,CR
1,2,9105801120601,10000.00,C,9841034054,"MOB,100003989053",1648192,2023-08-29,NaN,3989053,CR
2,3,9105801120601,20000.00,C,9816642631,"MOB,100003989010",1648092,2023-08-29,NaN,3989010,CR
3,4,9105801120601,15000.00,C,9851086526,"MOB,100003988851",1647903,2023-08-29,NaN,3988851,CR
4,5,9105801120601,200.00,C,9748361397,"MOB,100003988734",1647700,2023-08-29,NaN,3988734,CR
5,6,9105801120601,30940.00,D,LAXMIBANK#cash,020447,NPS-IF-3142883Off,2023-08-29,NaN,3142883,DR
6,7,9105801120601,1000.00,C,9762525844,"MOB,100003988698",1647664,2023-08-29,NaN,3988698,CR
7,8,9105801120601,200166.95,D,PRABHUMONEY#PMTFT23,020423,NPS-IF-3136924,2023-08-29,NaN,3136924,DR
8,9,9105801120601,10087.00,D,PRABHUMONEY#PMT1138,020422,NPS-IF-3136891,2023-08-29,NaN,3136891,DR
9,10,9105801120601,50000.00,C,9842100514,"MOB,100003988389",1647160,2023-08-29,NaN,3988389,CR
